Import modules.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym
from IPython.display import display, Markdown

# Suppress the use of scientific notation when printing small numbers
np.set_printoptions(suppress=True)

Define and linearize EOMs.

In [18]:
# Variables
q, v, tau = sym.symbols('q, v, tau')

# Constants
c_1 = sym.nsimplify(1.)
c_2 = sym.nsimplify(2.)

# Nonlinear EOMs
f = sym.simplify(sym.Matrix([v, (1 / c_1) * tau - (c_2 / c_1) * sym.sin(q)]))

# Equilibrium point
q_e = 0.
v_e = 0.
tau_e = float(c_2) * np.sin(q_e)

# Linearization
# - Take derivatives
A_sym = f.jacobian([q, v])
B_sym = f.jacobian([tau])
# - Create lambda functions so we can plug numbers into derivatives
A_num = sym.lambdify((q, v, tau), A_sym)
B_num = sym.lambdify((q, v, tau), B_sym)
# - Evaluate lambda functions at equilibrium point
A = A_num(q_e, v_e, tau_e).astype(float)
B = B_num(q_e, v_e, tau_e).astype(float)


Show results.

In [ ]:
# Show f
display(Markdown(fr'$$ f(m, n) = \begin{{align*}}{sym.latex(f)}\end{{align*}} $$'))

# Show A and B
A_str = sym.latex(sym.Matrix(A.round(decimals=4)))
B_str = sym.latex(sym.Matrix(B.round(decimals=4)))
display(Markdown(fr'$$ A = {A_str} \qquad\qquad B = {B_str} $$'))

Compare linear approximation to original nonlinear EOMs near equilibrium point.

In [ ]:
# Get a range of values
qs = np.linspace(q_e - (np.pi / 2), q_e + (np.pi / 2), 100)
vs = v_e * np.ones_like(qs)
taus = tau_e * np.ones_like(qs)

f_num = sym.lambdify((q, v, tau), f)
m_dot = f_num(qs, vs, taus)

x_dot = []
for (q_i, v_i, tau_i) in zip(qs, vs, taus):
    x = np.array([q_i - q_e, v_i - v_e])
    u = np.array([tau_i - tau_e])
    x_dot.append(A @ x + B @ u)
x_dot = np.array(x_dot)

plt.plot(qs, m_dot[1, 0, :], label='nonlinear')
plt.plot(qs, x_dot[:, 1], label='linear')
plt.plot(q_e, f_num(q_e, v_e, tau_e)[1], '.', markersize=12)
plt.xlabel(r'$q$')
plt.ylabel(r'$\dot{v}$')
plt.legend()
plt.grid()
plt.show()